In [1]:
# Required imports:
import numpy as np
import sympy as sp

from devito import *
from examples.seismic import plot_velocity, RickerSource, TimeAxis, Receiver, plot_image, AcquisitionGeometry
from model_VTI import ModelElasticVTI
from operators import *
from wavesolver import *

import matplotlib.pyplot as plt

from sympy import init_printing, latex
init_printing(use_latex='mathjax')

%matplotlib inline

plt.rc('font', size=14)

# default language set to openmp i.e. Code Generated by Devito will be in openmp
#configuration['language'] = 'openmp'

# logging set to debug to capture statistics on the performance of operators
configuration['log-level'] = 'DEBUG'

In [2]:
# define precision type: 32bit floating point
dtype = np.float32

#Thomsen's anisotropy parameters
epsilon = 0.25
delta = 0.10
gamma = 0.05

#initial conditions
npad = 40
vp = 2.0 #km/s
vs = 1.0 #km/s
rho= 1.8 #kg/m**3

In [3]:
# Model with fixed time step value
class ModelBench(ModelElasticVTI):
    """
    Physical model used for accuracy benchmarking.
    The critical dt is made small enough to ignore
    time discretization errors
    """

    @property
    def critical_dt(self):
        """Critical computational time step value."""
        return .1

In [4]:
# Discretization order
orders = (2, 4, 6, 8, 10)
norder = len(orders)

In [5]:
# Number of time steps
nt = 2001
# Time axis defined
dt = 0.1
t0 = 0.
tn = dt * (nt-1)
time = np.linspace(t0, tn, nt)
# Source peak frequency in KHz
fpeak = 0.09
f0 = fpeak

print("Time Start: ", t0 )
print("Time End: ", tn )
print("Time Step: ", dt )
print("Number of Time Steps: ", nt )
print("Source Peak Frequency: ", f0 )

Time Start:  0.0
Time End:  200.0
Time Step:  0.1
Number of Time Steps:  2001
Source Peak Frequency:  0.09


In [6]:
# Increasing grid spacing and Domain size
sizes = ((201, 2.0), (161, 2.5), (101, 4.0))
dx = [2.0, 2.5, 4.0]
nsizes = len(sizes)

In [7]:
# Fine grid model
c0 = 1.5
model = ModelBench(vp=c0, vs=vs, rho=rho, epsilon=epsilon, delta=delta, gamma=gamma, origin=(0., 0., 0.), spacing=(.5, .5, .5),
bcs="damp", shape=(201, 201, 201), space_order=20, nbl=npad, dtype=np.float32)

Operator `initdamp` generated in 0.17 s
  * lowering.Clusters: 0.07 s (42.8 %)
     * specializing.Clusters: 0.05 s (30.6 %)
  * lowering.Expressions: 0.05 s (30.6 %)
  * lowering.IET: 0.05 s (30.6 %)
Flops reduction after symbolic optimization: [102 --> 102]
Allocating memory for damp(283, 283, 283)
Operator `initdamp` fetched `/var/folders/kb/bc2lwpvd4kj2k3qjll_cmwp80000gn/T/devito-jitcache-uid501/624b4455001f27d77e8620a18c0893867bdf4357.c` in 0.08 s from jit-cache
Operator `initdamp` run in 0.14 s
* section0<<40,281,281>,<40,281,281>,<281,40,281>,<281,40,281>,<281,281,40>,<281,281,40>> with OI=0.01 computed in 0.14 s [2.43 GFlops/s]
Performance[mode=advanced] arguments: {}


In [8]:
# Source and receiver geometries
src_coordinates = np.array([100., 100., 100.])

# Single receiver offset 100 m from source
rec_coordinates = np.array([150., 150., 5.])


print("The computational Grid has (%s, %s, %s) grid points "
       "and a physical extent of (%sm, %sm, %sm)" % (*model.grid.shape, *model.grid.extent))
print("Source is at the center with coordinates", src_coordinates)
print("Receiver (single receiver) is located at", rec_coordinates)
    
# Note: gets time sampling from model.critical_dt
geometry = AcquisitionGeometry(model, rec_coordinates, src_coordinates, 
                                   t0=t0, tn=tn, src_type='Ricker', f0=f0, t0w=1.5/f0)                     

The computational Grid has (281, 281, 281) grid points and a physical extent of (140.0m, 140.0m, 140.0m)
Source is at the center with coordinates [100. 100. 100.]
Receiver (single receiver) is located at [150. 150.   5.]


In [9]:
solver = ElasticVTIWaveSolver(model, geometry, space_order=4)
ref_rec, ref_u, _ = solver.forward()

Allocating memory for src_coords(3, 3)
Allocating memory for src(2001, 3)
Allocating memory for rec1_coords(3, 3)
Allocating memory for rec2_coords(3, 3)
